In [1]:
import os
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from PIL import Image
import numpy as np

In [1]:
dataset = datasets.ImageFolder('D:/competition/10 Monkey Species/data/training/',
                        transform=transforms.Compose([transforms.Resize((256, 256), interpolation=Image.NEAREST),
                        transforms.Grayscale(num_output_channels=1), 
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.5], std=[0.5])]))
        
lengths = [int(np.ceil(0.9*len(dataset))), int(np.floor(0.1*len(dataset)))]

#Dividing into validation and training set
train_set, val_set = random_split(dataset, lengths)

NameError: name 'datasets' is not defined

In [6]:
len(val_set)

109

In [2]:
class smallAndSmartModel(pl.LightningModule):
    def __init__(self, num_classes=10):
        super(smallAndSmartModel, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1, 28, kernel_size=5),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2))
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(28, 10, kernel_size=3),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2))
        self.dropout1 = torch.nn.Dropout(0.25)
        self.fc1 = torch.nn.Linear(38440, 18)
        self.dropout2 = torch.nn.Dropout(0.08)
        self.fc2 = torch.nn.Linear(18, num_classes)
    
    
    def train_dataloader(self):
        dataset = datasets.ImageFolder('D:/competition/10 Monkey Species/data/training/',
                        transform=transforms.Compose([transforms.Resize((256, 256), interpolation=Image.NEAREST),
                        transforms.Grayscale(num_output_channels=1), 
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.5], std=[0.5])]))
        
        lengths = [int(np.ceil(0.9*len(dataset))), int(np.floor(0.1*len(dataset)))]
        
        #Dividing into validation and training set
        self.train_set, self.val_set = random_split(dataset, lengths)
        
        self.test_set = datasets.ImageFolder('D:/competition/10 Monkey Species/data/validation/',
                        transform=transforms.Compose([transforms.Resize((256, 256), interpolation=Image.NEAREST),
                        transforms.Grayscale(num_output_channels=1), 
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.5], std=[0.5])]))
        return DataLoader(self.train_set, batch_size=64)
        
    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(self.val_set, batch_size=64)

    def test_dataloader(self):
        # OPTIONAL
        return DataLoader(self.test_set, batch_size=64)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.dropout1(x)
        #view is the flatten step
        x = torch.relu(self.fc1(x.view(x.size(0), -1)))
        x = F.leaky_relu(self.dropout2(x))
        return F.softmax(self.fc2(x))

    def configure_optimizers(self):
        # Essential fuction
        #we are using Adam optimizer for our model
        return torch.optim.Adam(self.parameters())
    
    def training_step(self, batch, batch_idx):
        #extracting input and output from the batch
        x, labels = batch
        #doing a forward pass
        pred = self.forward(x)
        #calculating the loss
        loss = F.nll_loss(pred, labels)
        
        acc = accuracy(pred, labels)
        pbar = {"train_acc": acc}
        #logs
        logs={"train_loss": loss, "train_acc": acc}  
        self.log('train_loss', loss)
        output={
              #REQUIRED: It ie required for us to return "loss"
              "loss": loss,
              #optional for logging purposes
              "log": logs
          }
        return output
    
    def test_step(self, data_batch, batch_nb):
        x, y = data_batch

        # implement your own
        out = self.forward(x)
        loss = F.nll_loss(out, y)

        # calculate acc
        labels_hat = torch.argmax(out, dim=1)
        test_acc = torch.sum(y == labels_hat).item() / (len(y) * 1.0)

        # all optional...
        # return whatever you need for the collation function test_end
        output = OrderedDict({
            'test_loss': loss_test,
            'test_acc': torch.tensor(test_acc), # everything must be a tensor
        })

        # return an optional dict
        return output

In [3]:
myTrainer = pl.Trainer(gpus=1, max_epochs=10)

model = smallAndSmartModel()
myTrainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\arvin\miniconda3\envs\gpu\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name     | Type       | Params
----------------------------------------
0 | layer1   | Sequential | 728   
1 | layer2   | Sequential | 2.5 K 
2 | dropout1 | Dropout    | 0     
3 | fc1      | Linear     | 691 K 
4 | dropout2 | Dropout    | 0     
5 | fc2      | Linear     | 190   
----------------------------------------
695 K     Trainable params
0         Non-trainable params
695 K     Total params


Training: 0it [00:00, ?it/s]

C:\Users\arvin\miniconda3\envs\gpu\lib\site-packages\ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\arvin\miniconda3\envs\gpu\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
C:\Users\arvin\miniconda3\envs\gpu\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [6]:
myTrainer.test(model, test_dataloaders=model.test_dataloader())

C:\Users\arvin\miniconda3\envs\gpu\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: you passed in a test_dataloader but have no test_step. Skipping test loop
  warnings.warn(*args, **kwargs)


1